In [43]:
!pip install biopython==1.81

In [44]:
## This code block imports python modules to work with during the lab, you need to run it first to get started.
# modules
import py3Dmol
import nglview
import os
import shutil
#from Bio.PDB import PDBParser, PDBIO, Select,  PDBList, MMCIFParser, StructureAlignment
from Bio.PDB import PDBList, MMCIFParser, Select, PDBIO
import Bio.Align
import os
from pathlib import Path
import rdkit

# local scripts
from scripts import viewer
from scripts import bio_align

# Assignments:
The goal of this course is to design a new ligand for the target that was assigned to you. You will work in pairs. This ligand should dock with a higher docking score than the co-crystalized ligand. You will reach this goal by executing a bio-informatics part, a chem-informatics part, and finally perform docking.

Measurement Report:
The results from your practical work are to be written down in a measurement report, which is to be handed in on Monday. This report should contain answers to the questions from the manual, and descriptions of what you did during the course (your train of thought). Naturally you should also include name, student number, etc. 

Presentation:
In addition, you are to hold a presentation on these subjects on Friday the 7th of October (afternoon). The presentation should last ~10 minutes with some subsequent discussion. In the presentation you should provide a background of your drug target and drug (type, pathology etc). There should be some results from the QSAR modeling and several docked poses. You should present your newly designed drug and defend why it is better. 

The practical consists of three parts, which each have their own Jupyter notebook with instructions and the code to run:

- Intro to cheminformatics
- Machine learning and QSAR
- Working with crystal structures



# Bioinformatics

### Analyzing your Drug target

It is important to get a good feeling for the target you are about to study. This is where literature research comes into play. A good starting point is the original publication that your PDB code refers to. This will also give you further references. In addition, general reviews (Pubmed, Google Scholar) can be very helpful.

1.	Describe the target (your report should contain at least the following information):
-	Pathology (why are we interested in the target?)
-	Clinical drugs available (which drugs are on the market)
 	Also use ATC information to further describe the drugs. 
-	Protein information (what family of proteins is the target a member off, where is it located in the body?)

### Related proteins (off-targets), based on sequence

Using Bio-informatics we are going to compare the target to other proteins. We use the primary structure (sequence) of the protein for this. Go to: http://www.uniprot.org/

2.	You can find the link to Uniprot on the sequence tab on the page of your target on https://www.rcsb.org/

3.	Alternatively, find the name of the target in the ‘Protein Knowledgebase’ (UniprotKB). From the list of hits you should select the correct one.

4.	Verify the ‘Accession’ number with your supervisor. Yellow stars means that it is a correct and reviewed target, use a reviewed one unless you are working on H5N1. 

In the case of the HIV targets, you have to make a sub selection within the POL polyprotein, do this by selecting the relevant domain in the list of domains below on the page.
RT -> select the ‘Reverse Transcriptase domain’, ‘Reverse Transcriptase Ribonuclease H’ domain. 
Protease -> select the ‘Protease domain’

5.	When you have found the correct one, copy the following information:
    -	Sequence Length / Status / Protein Existence
    -	Mass (kDa)

6.	Use BLAST (Basic Local Alignment Search Tool, under ‘Advanced’) to find identical proteins in other species (related target 1).
-	Database ‘UniProtKB’, Threshold at 10, Matrix on auto, Filtering on none, Gapped on yes and Hits on 1000. Copy the following information. 
    -	Length 
    -	Identity
    -	Species
    
7.	Click on the ‘Accession’ number of the same hit, write the ‘Accession’ number down and copy the following information: 
-	Status / Protein Existence
-	Mass (kDa)

8.	Use BLAST to find similar targets in the same species (related target 2).
-	Database ‘...Human’, Threshold at 1, Matrix on auto, Filtering on none, Gapped on yes and Hits on 1000. Copy the following information of the first hit that is not an isoform of your target.
    -	Length 
    -	Identity
9.	Click on the ‘Accession’ number of the same hit, write the ‘Accession’ number down and copy the following information: 
    -	Status / Protein Existence
    -	Mass (kDa)
    
Questions (1):
-	Which target is more similar compared to the original target?
-	Did you expect this?

Align the 3 proteins on Uniprot using the ‘Accession’ numbers. 

In the overview, select the following tickboxes: “Similarity, Hydrophobic, Negative, Positive, Aromatic”, scroll down and make a screenshot (for report and presentation). Which targets are ‘more similar’ ?

### Related proteins (off-targets), based on structure

Since we are in the fortunate position that we have a crystal structure, we are also going to use a 3D similarity search. This is a structural similarity search rather than a sequence similarity search. Now we compare proteins based on their tertiary structure (3D structure)

10.	Start at http://www.rcsb.org/pdb/home/home.do  and find your target using the PDB identifier. 
11.	Scroll down to ‘macromolecules’.
12.	Click ‘structure’

![image info](img/LAB00_FIG00.png)

13.	The top one should be your protein.
•	Write down the accession of the next most similar one 
•	In the case of a GPCR target, ignore the T4Lysozyme hits
14.	Go to: https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html use align, protein complexes, 2 PDB structures

![image info](img/LAB00_FIG01.png)

15.	Input both pdb codes and click all matching molecules superposed.

16.	After completion do file  realign and select ‘on sequence alignment’ 

17.	Select the ‘A’ chains and click realign. Note the RMSD below (realignment RMSD). 

![image info](img/LAB00_FIG02.png)

18.	Make a screenshot of the superposition. 

# Retrieving a 3D structure

Next, we will prepare our protein. The first step is to download it from the .pdb. We will the ngl viewer for this. 



We will add hydrogen atoms to the structure, because these are normally not resolved in the structure (due to limitations in resolution of the experimental method.

19. First, we make a working directory in your home directory:

In [45]:
HOMEDIR = str(Path.home())
os.chdir(HOMEDIR)
# We need to check whether the directory is there
try:
    os.mkdir('Bioinformatics')
except:
    print("Directory already exists")
os.chdir('Bioinformatics')

Directory already exists


Next, you need to specify the PDB code of your protein. We have used 4eiy as an example. If you execute the code block below, you will see a viewer with a 3D structure of the protein.

In [46]:
TARGET_PDB_ID = "5F1A" # #5F1A is COX2+ SAL
view = nglview.show_pdbid(TARGET_PDB_ID)
view.center()
view

NGLWidget()

It  is nice to have the overview of the structure, but since we are interested in designing new drugs, it makes more sense to have a closer look at the co-crystallized ligand. For this, we need to know the residue name of the ligand. This is a three later amino acid code that you can retrieve from the RCSB. In this case, the amino acid code is ZMA. First we zoom in on the ligand.

In [47]:
LIGAND_CODE = "SAL" # SAL is cocrystallized ligand of 5F1A, RCX is Rofecoxib

view.center(LIGAND_CODE)
view

NGLWidget()

Next, let's show the residues that are within 5 Angstrom of the ligand

In [48]:
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

NGLWidget()

Have a look at the residues near the ligand, can you observe any important interactions? Describe in your report which interactions you observe, and what type of interactions they are.

Note that we do not see any hydrogen atoms, do you know why?

In the next stage, we will add the hydrogens and have another look at the structure. We will split the protein and ligand and save them seperately. For this, we use biopython. https://biopython.org/docs/1.75/api/Bio.html

First download the coordinates from RCSB.

In [49]:
pdbl = PDBList()
pdbl.retrieve_pdb_file(TARGET_PDB_ID, pdir=TARGET_PDB_ID)

Structure exists: '5F1A/5f1a.cif' 


'5F1A/5f1a.cif'

Next, we generate a BioPython object from the coordinates, which we can use for various tasks.

In [50]:
parser = MMCIFParser()
structure = parser.get_structure("TARGETPROT",'{}/{}.cif'.format(TARGET_PDB_ID,TARGET_PDB_ID.lower()))

/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9002.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9086.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9205.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9417.
  warnings.warn(


Now we save the ligand

In [51]:
class ResSelect(Select):
    def accept_residue(self, residue):
        if residue.get_resname() == LIGAND_CODE:
            return 1
        else:
            return 0

class NonHetSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.id[0] == " " else 0

io = PDBIO()
io.set_structure(structure)
io.save("ligand-{}.pdb".format(LIGAND_CODE), ResSelect())
io.save("protein-{}.pdb".format(TARGET_PDB_ID), NonHetSelect())

Now that we have seperated ligands and protein, we can start adding hydrogens to the protein, we will use LePro for this, which is part of the LeDock program (https://en.wikipedia.org/wiki/LeDock).

21. Prepare the protein:

In [52]:
command = '../CBR_teaching/bin/lepro protein-{}.pdb'.format(TARGET_PDB_ID)
os.system(command)
shutil.move('pro.pdb','{}_prepped.pdb'.format(TARGET_PDB_ID))

'5F1A_prepped.pdb'

In [53]:
# combine protein and ligand files
filenames = [
'{}_prepped.pdb'.format(TARGET_PDB_ID),
"ligand-{}.pdb".format(LIGAND_CODE)
]
with open('{}-complex.pdb'.format(TARGET_PDB_ID), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

In [54]:
with open('{}-complex.pdb'.format(TARGET_PDB_ID)) as f:
    view = nglview.show_file(f, ext="pdb")
    
view.center(LIGAND_CODE)
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

NGLWidget()

In [55]:
# Geript van 03-Docking, variable names veranderd

view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('{}_prepped.pdb'.format(TARGET_PDB_ID),'r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

ligtmp = 'ligand-{}.pdb'.format(LIGAND_CODE)
view.addModels(open(ligtmp,'r').read(),format='pdb')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Let's have a look again at the protein

You can move around the structure with the mouse. Scrolling will zoom in/out, with the left mouse button you can rotate the structure. If you click the scrolling wheel you can move around the structure (sliding).

23. Now, we will repeat the procedure for the most similar target that you identified (the highest scoring hit from the PDB):

In [56]:
OFF_TARGET_PDB_ID = "4O1Z" # Enter your off target PDB code here, example = '5uen'
OFF_TARGET_LIGAND = "MXM"  # Enter the ligand code here, example = 'DU1'

pdbl = PDBList()
pdbl.retrieve_pdb_file(OFF_TARGET_PDB_ID, pdir=OFF_TARGET_PDB_ID)

parser = MMCIFParser()
structure = parser.get_structure("TARGETPROT",'{}/{}.cif'.format(OFF_TARGET_PDB_ID,OFF_TARGET_PDB_ID.lower()))

class ResSelect(Select):
    def accept_residue(self, residue):
        if residue.get_resname() == OFF_TARGET_LIGAND:
            return 1
        else:
            return 0

io = PDBIO()
io.set_structure(structure)
io.save("ligand-{}.pdb".format(OFF_TARGET_LIGAND), ResSelect())
io.save("protein-{}.pdb".format(OFF_TARGET_PDB_ID), NonHetSelect())

Structure exists: '4O1Z/4o1z.cif' 


/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9096.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9199.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9302.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9463.
  warnings.warn(


In [57]:
command = '../CBR_teaching/bin/lepro protein-{}.pdb'.format(OFF_TARGET_PDB_ID)
os.system(command)
shutil.move('pro.pdb','{}_prepped.pdb'.format(OFF_TARGET_PDB_ID))

'4O1Z_prepped.pdb'

In [58]:
# combine protein and ligand files
filenames = [
'{}_prepped.pdb'.format(OFF_TARGET_PDB_ID),
"ligand-{}.pdb".format(OFF_TARGET_LIGAND)
]
with open('{}-complex.pdb'.format(OFF_TARGET_PDB_ID), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

24. Prepare the protein

In [59]:
with open('{}-complex.pdb'.format(OFF_TARGET_PDB_ID)) as f:
    view = nglview.show_file(f, ext="pdb")
    
view.center(OFF_TARGET_LIGAND)
viewer.show_residues_around(view, selection=OFF_TARGET_LIGAND)
view

NGLWidget()

25. Now, let's try to align the structures. First we generate the alignment object

In [60]:
from Bio import pairwise2
from Bio.Seq import Seq 
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

# Get the structures
PDBCODE_1 = '5F1A' # Name of the first structure
PDBCODE_2 = '4O1Z' # Name of the second structure

import requests
data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{PDBCODE_1}').json()[PDBCODE_1.lower()]
SEQ1 = (data[0]['sequence'])
SEQ1 = Seq(SEQ1)

data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{PDBCODE_2}').json()[PDBCODE_2.lower()]
SEQ2 = (data[0]['sequence'])
SEQ2 = Seq(SEQ2)

alignments = pairwise2.align.globalxx(SEQ1, SEQ2)

for align1, align2, score, begin, end in alignments:
    filename = "alignment.fasta"
    with open(filename, "w") as handle:
        handle.write(">SEQ1\n%s\n>SEQ2\n%s\n" % (align1, align2))

print(alignments[0])

Alignment(seqA='K--NPCCSH--PCQNR--GV-CMSVG-F--DQ-YK-CDCTRTGFY-GE-NCST-PEFLTRIK---LF---LK-PT-PNTVHY--I--L-THFKGF-WNVVNNIPFLR----NA--I----MSY--VLTS-RSH-LID-SPPTYN-A-DYGYK-SWEA-FSNL-SYYTRA-LPP-VPD-DCPTPL-GV-KGKKQLPDSNEIV-EK-L----LLRRKFIPDPQGS-NM-MFAFFAQHFTHQFFKTDH--KR-GPA-FTNG--LGHGVDLN-HIYGET--LA-RQRK--LRLFKDGKM-KYQIID---GEM-YPPT-VKDTQAE-----MI-YP---PQVPEHLRF----AVGQEVFGLV-PGLMM-YATIWLREHNRVCDVL-KQ-EHPE-WGDEQLFQTS-RLILIGETIKIVIED-YVQH-LSGYHFKL--KFDPELLFNK--QFQYQ-NRIAA-EFNT-LYHWHPLL-PDT-F----QIHDQKYN-YQ-QFIYN--N-SIL-LEH---GITQFVES-----FT-RQI-AGRVA-GG-RN-------VPPAVQK-VSQASIDQ--SRQMKY----QS-FNEYRKRF-MLKPYE-SFE-ELTGEKEMSA-ELEA-LYGDIDAVEL--YPA-LLV-EKPR--PDA--IFGET-MV-EV-GAPFSLKGLM-GNV-ICSPA-YWKP-STFGGEVGFQIIN---TASIQS-LI----CN-NV-KG-CPFT--SFS-VPD----------------', seqB='-PVNPCC--YYPCQ--HQG-IC--V-RFGLD-RY-QCDCTRTG-YSG-PNC-TIPE----I-WTWL-RTTL-RP-SP-----SFIHFLLTH--G-RW--------L-WDFVNATFIRDTLM--RLVLT-VRS-NLI-PSPPTYNIAHD--Y-ISWE-SFSN-VSYYTR-IL-PSVP-RDCPTP-MG-TKGKKQLPD-----AE-FLSRRFLLRRKFIPDPQG-TN

You can also align structure using https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html as you've seen previously.

You can also have a look at PLIP, which is a nice interaction profiler as well:
https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index

Finally, the interaction viewer on rcsb itself is also quite good, just type in your accession code in the search bar, scroll down to the ligand overview and click the ligand interaction box.

Toy around a bit with the structures and see if you can get a feeling for the binding site. In particular what kind of interactions seem to be important. Feel free to discuss this with your supervisor as well!

Another great way to look at the structure is on the rcsb website itself, where you can go to a 3D viewer of the ligand binding site by clicking on the Ligand interaction button. 



This is the end of the bioinformatics part of the lab, for the report, please add a screenshot of the PLIP viewer. Try to identify key interactions in the binding site, either on the RCSB website or PLIP. Also think already how you might improve the ligand, can you think of additional interactions? The next two days, we will try to optimize the ligand. Tomorrow we will work with QSAR and cheminformatics. The day after we will use docking